In [121]:
import pandas as pd
from matplotlib import pyplot as plt

In [98]:
df_m_metadata=pd.read_csv('movie.metadata.tsv', sep='\t')
#il y a bcp plus de movies metadata que de plots. 2x plus

In [102]:
df_m_metadata.columns = ["wikipedia_m_ID", "freebase_m_ID", "m_name", "m_relase_date", "m_BO_revenue", "m_runtime", "m_languages", "m_countries", "m_genres"]
df_m_metadata_reseted_index=df_m_metadata.set_index('wikipedia_m_ID')
df_m_metadata.head()

,wikipedia_m_ID,freebase_m_ID,m_name,m_relase_date,m_BO_revenue,m_runtime,m_languages,m_countries,m_genres
0,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
1,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
2,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
3,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"
4,13696889,/m/03cfc81,The Gangsters,1913-05-29,NaN,35.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""Eng...","{""/m/09c7w0"": ""United States of America""}","{""/m/02hmvc"": ""Short Film"", ""/m/06ppq"": ""Silen..."


In [192]:
df_genre_exploded=df_m_metadata_reseted_index[['m_genres']]
df_genre_exploded['m_genres']=df_genre_exploded.apply(lambda row: list(row['m_genres'].strip('{}').replace('"', '').split(",")), axis=1)
df_genre_exploded=df_genre_exploded.explode('m_genres')
df_genre_exploded[['m_genres_freebase_ID', 'm_genres']]=df_genre_exploded['m_genres'].str.split(': ',2 , expand=True)
df_genre_exploded.reset_index(inplace=True)

df_countries_exploded=df_m_metadata_reseted_index[['m_countries']]
df_countries_exploded['m_countries']=df_countries_exploded.apply(lambda row: list(row['m_countries'].strip('{}').replace('"', '').split(",")), axis=1)
df_countries_exploded=df_countries_exploded.explode('m_countries')
df_countries_exploded[['m_countries_freebase_ID', 'm_countries']]=df_countries_exploded['m_countries'].str.split(': ',2 , expand=True)

df_languages_exploded=df_m_metadata_reseted_index[['m_languages']]
df_languages_exploded['m_languages']=df_languages_exploded.apply(lambda row: list(row['m_languages'].strip('{}').replace('"', '').split(",")), axis=1)
df_languages_exploded=df_languages_exploded.explode('m_languages')
df_languages_exploded[['m_languages_freebase_ID', 'm_languages']]=df_languages_exploded['m_languages'].str.split(': ',2 , expand=True)


C:\Users\james\AppData\Local\Temp\ipykernel_20732\3908948819.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genre_exploded['m_genres']=df_genre_exploded.apply(lambda row: list(row['m_genres'].strip('{}').replace('"', '').split(",")), axis=1)
C:\Users\james\AppData\Local\Temp\ipykernel_20732\3908948819.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_countries_exploded['m_countries']=df_countries_exploded.apply(lambda row: list(row['m_countries'].strip('{}').replace('"', '').split(",")), axis=1

In [193]:
pd.set_option('display.max_rows',None)
df_genre_exploded['m_genres'].value_counts().sort_index()

Absurdism                                      92
Acid western                                    9
Action                                       8797
Action Comedy                                 169
Action Thrillers                              516
Action/Adventure                             4561
Addiction Drama                                61
Adult                                         223
Adventure                                    4962
Adventure Comedy                              144
Airplanes and airports                         56
Albino bias                                    17
Alien Film                                    113
Alien invasion                                  6
Americana                                     107
Animal Picture                                183
Animals                                        49
Animated Musical                               54
Animated cartoon                              122
Animation                                    3534


In [194]:
df_countries_exploded['m_countries'].value_counts().sort_index()

Afghanistan                                    15
Albania                                        28
Algeria                                        43
Argentina                                    1468
Armenia                                        16
Aruba                                           3
Australia                                    1114
Austria                                       267
Azerbaijan                                     25
Bahamas                                         4
Bahrain                                         3
Bangladesh                                     71
Belgium                                       360
Bhutan                                          4
Bolivia                                        15
Bosnia and Herzegovina                         35
Brazil                                        298
Bulgaria                                      105
Burkina Faso                                   35
Burma                                          24


In [196]:
df_languages_exploded['m_languages'].value_counts().sort_index()

Aboriginal Malay languages                                                      2
Aceh Language                                                                   1
Afrikaans Language                                                             41
Akan Language                                                                   2
Albanian language                                                              43
Algonquin Language                                                              2
American English                                                              114
American Sign Language                                                         33
Amharic Language                                                               11
Ancient Greek                                                                   2
Apache                                                                          2
Arabic Language                                                               499
Aramaic language